In [1]:
import numpy as np
import random

BOARD_SIZE = 10

In [2]:
# La lógica sería 0 para agua, 1 para barco intacto, -1 para barco dañado y 2 para disparo fallido (en el agua)
def turno(jugador, tablero_oponente, disparos_jugador_a=None):
    while True:
        if jugador == "A":
            if not disparos_jugador_a:
                print("Jugador A no tiene más disparos.")
                return False
            x, y = disparos_jugador_a.pop(0)  # coordenadas importadas
        else:  # Jugador B en aleatorio
            x, y = random.randint(0, BOARD_SIZE-1), random.randint(0, BOARD_SIZE-1)
            print(f"Jugador B dispara a ({x},{y})")

        # Comprobar que sean válidas
        if not (0 <= x < BOARD_SIZE and 0 <= y < BOARD_SIZE):
            print("Coordenadas fuera del tablero.")
            continue

        # Lógica de disparo
        if tablero_oponente[x, y] == 1:
            tablero_oponente[x, y] = -1
            print(f"{jugador} acertó en ({x},{y}) y repite turno.")
            if not condicion_victoria(tablero_oponente):
                print(f"Victoria de {jugador}!")
                return True
            continue  # repite turno
        elif tablero_oponente[x, y] == 0:
            tablero_oponente[x, y] = 2
            print(f"{jugador} falló en ({x},{y}). Turno del otro jugador.")
            return False
        else:
            print(f"{jugador} ya disparó en ({x},{y}). Elige otras coordenadas.")
            continue

In [3]:
def condicion_victoria(tablero_oponente):
    return np.any(tablero_oponente == 1)

In [4]:
# Estos son datos que le puse pa probar.
# Tablero del jugador B con barcos
tablero_b = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
tablero_b[2, 3] = 1
tablero_b[5, 5] = 1

# Disparos del jugador A (importados de otra función)
disparos_a = [(2, 3), (0, 0), (5, 5)]

# Simulación
while disparos_a and condicion_victoria(tablero_b):
    turno("A", tablero_b, disparos_a)

A acertó en (2,3) y repite turno.
A falló en (0,0). Turno del otro jugador.
A acertó en (5,5) y repite turno.
Victoria de A!


In [5]:
# Importar archivos
from variables import *
from clases import *
import numpy as np
import random
from funciones import *



# ==============================
# PROGRAMA PRINCIPAL
# ==============================
while True:
    print("=== ¡Bienvenido a Hundir la Flota! ===")
    print("Reglas básicas:")
    print("- Tablero de 10x10.")
    print("- Dispara introduciendo fila y columna (0–9).")
    print("- Si aciertas, vuelves a disparar. Si fallas, dispara la máquina.\n")
    
    nombre = input("Introduce tu nombre: ")

    # Crear tableros (de variables.py)
    tablero_jugador = crear_tablero()
    tablero_rival = crear_tablero()
    tablero_rival_visible = crear_tablero()

    # ==============================
    # COLOCAR BARCOS DEL JUGADOR
    # ==============================
    print(f"\nHola {nombre}, coloca tus barcos en el tablero.")
    print("Tienes:")
    print("- 4 barcos de 1 posición")
    print("- 3 barcos de 2 posiciones")
    print("- 2 barcos de 3 posiciones")
    print("- 1 barco de 4 posiciones")

    flota_jugador = []

    # 4 barcos pequeños
    for i in range(4):
        coords = colocar_barco_jugador(tablero_jugador, 1, "barco pequeño", "P")
        flota_jugador.append(coords)

    # 3 barcos medianos
    for i in range(3):
        coords = colocar_barco_jugador(tablero_jugador, 2, "barco mediano", "M")
        flota_jugador.append(coords)

    # 2 barcos grandes
    for i in range(2):
        coords = colocar_barco_jugador(tablero_jugador, 3, "barco grande", "G")
        flota_jugador.append(coords)

    # 1 barco gigante
    coords = colocar_barco_jugador(tablero_jugador, 4, "barco gigante", "E")
    flota_jugador.append(coords)

    # Mostrar flota
    

    print("\n🛳️ Tu tablero queda así:\n")
    mostrar_tablero(tablero_jugador, ocultar_barcos=False)
    mostrar_tablero(tablero_rival_visible, ocultar_barcos=True)

    # ==============================
    # COLOCAR BARCOS DEL RIVAL
    # ==============================
    print("\nColocando barcos del rival aleatoriamente...")

    flota_peq, tablero_rival = flota_peq_aleatorio(tablero_rival)
    flota_med, tablero_rival = flota_med_aleatoria(tablero_rival, flota_peq)
    flota_grand, tablero_rival = flota_grand_aleatoria(tablero_rival, flota_peq, flota_med)
    flota_gigante, tablero_rival = flota_enorme_aleatoria(tablero_rival, flota_peq, flota_med, flota_grand)

    turno_jugador = True

    # ==============================
    # BUCLE DE PARTIDA
    # ==============================
    while True:
         # Tu tablero con todos tus barcos visibles
        print("\nTu tablero:")
        mostrar_tablero(tablero_jugador, ocultar_barcos=False)

        # Tablero enemigo: sin mostrar barcos, solo X y O
        print("\nTablero enemigo visible:")
        mostrar_tablero(tablero_rival_visible, ocultar_barcos=True)

        if turno_jugador:
            print(f"\nTurno de {nombre}:")
            try:
                fila, col = map(int, input("Introduce las coordenadas para disparar (fila,col): ").split(","))
            except:
                print("Formato incorrecto. Usa fila,col (ejemplo: 3,5).")
                continue

            resultado = disparar(tablero_rival, tablero_rival_visible, fila, col)

            if resultado is True:
                print("🎯 ¡Has acertado! Vuelves a disparar.")
            elif resultado is False:
                print("🌊 Has fallado. Le toca a la máquina.")
                turno_jugador = False
            else:
                print("⚠️ Esa posición ya fue atacada, el turno pasa igualmente.")
                turno_jugador = False

            if comprobar_derrota(tablero_rival):
                print(f"🎉 ¡{nombre} ha ganado! Todos los barcos enemigos han sido hundidos.")
                break

        else:
            print("\nTurno de la máquina:")
            resultado = disparo_rival(tablero_jugador)

            if resultado:
                print("💥 La máquina ha acertado y repite turno.")
                turno_jugador = False
            else:
                print("💧 La máquina ha fallado. Te toca a ti.")
                turno_jugador = True

            if comprobar_derrota(tablero_jugador):
                print("💀 ¡La máquina ha ganado! Todos tus barcos han sido hundidos.")
                break

    # ==============================
    # FIN DE PARTIDA / REINICIO
    # ==============================
    print("\nPartida terminada.")
    calcular_estadisticas(tablero_jugador)
    calcular_estadisticas(tablero_rival)

    opcion = input("¿Quieres jugar otra vez? (s/n): ").lower()
    if opcion != "s":
        print("¡Gracias por jugar! Hasta la próxima.")
        break
    else:
        print("\nReiniciando partida...\n")


ModuleNotFoundError: No module named 'variables'

In [4]:
import pandas as pd
import numpy as np
import random 
BARCO = 'B'

In [5]:
def flota_peq_aleatorio(tablero):
    # Dimensiones de tablero:
    filas_totales = tablero.shape[0]
    columnas_totales = tablero.shape[1] 
    # Fijamos unos numeros random que tendrá la computadora.
    random.seed(42)

    flota_peq = [] 
    ocupadas = set() # Set para guardar las posiciones y que nos servira a futuro de comprobar que no estemos poniendo en la misma posicion 2 barcos.

    for barco in range(4):
        while True:
            fila = random.randint(0,filas_totales-1)
            columna = random.randint(0,columnas_totales-1)
            primera_pieza = (fila,columna)

            if primera_pieza in ocupadas:
                continue

            flota_peq.append(primera_pieza)
            ocupadas.add(primera_pieza)
            tablero[primera_pieza] = BARCO

            break

    return flota_peq, tablero






def flota_med_aleatoria(tablero, flota_peq):
    filas_totales = tablero.shape[0]
    columnas_totales = tablero.shape[1] 
    
    ocupadas = set(flota_peq)
    flota_med=[]
    random.seed(42)
    for barco in range(3):
        while True:
        # Primera pieza:
            fila = random.randint(0,filas_totales-1)
            columna = random.randint(0,columnas_totales-1)
            primera_pieza = (fila,columna)
        # Si la primera pieza es una de las ocupadas, vuelve a sacar una primera pieza
            if primera_pieza in ocupadas:
                continue
        # Elegimos orientacion, para asegurarnos que esten juntas las coordenadas de los barcos medianos.            
            orientacion = random.choice(['N','S','O','E'])

            fila2,columna2 = fila,columna # Muy importante ya que le decimos que la fila y columna de la segunda pieza en principio sea igual que la primera pieza y en funcion de la coordenad, sumamos o restamos columna o fila.

            match orientacion:
                case 'N':
                    fila2 = fila - 1
                                
                case 'S':
                    fila2 = fila + 1
                   
                case 'O':
                   columna2 = columna - 1
                    
                case 'E':
                    columna2 = columna + 1
                   
            if not (0 <= fila2 < filas_totales and 0 <= columna2 < columnas_totales):
                continue # Si la fila y columna no es mayor que las totlas y no son menores que 0, agregala a la variable segunda_pieza
            
            segunda_pieza = (fila2,columna2)

            if segunda_pieza in ocupadas:
                continue

            flota_med.append([primera_pieza,segunda_pieza])
            ocupadas.add(primera_pieza)
            ocupadas.add(segunda_pieza)

            tablero[primera_pieza] = BARCO
            tablero[segunda_pieza] = BARCO



            break
    return flota_med,tablero




def flota_grand_aleatoria(tablero, flota_peq,flota_med):
    filas_totales = tablero.shape[0]
    columnas_totales = tablero.shape[1] 

    ocupadas = set(flota_peq) 
    for barco in flota_med:
        for pieza in barco:
            ocupadas.add(pieza)
    flota_grand = []
    random.seed(42)
    for barco in range(2):
        while True:
        # Primera pieza:
            fila = random.randint(0,filas_totales-1)
            columna = random.randint(0,columnas_totales-1)
            primera_pieza = (fila,columna)
        # Si la primera pieza es una de las ocupadas, vuelve a sacar una primera pieza
            if primera_pieza in ocupadas:
                continue
        # Elegimos orientacion, para asegurarnos que esten juntas las coordenadas de los barcos medianos.            
            orientacion = random.choice(['N','S','O','E'])

            fila2,columna2 = fila,columna
            fila3,columna3 = fila,columna  # Muy importante ya que le decimos que la fila y columna de la segunda pieza en principio sea igual que la primera pieza y en funcion de la coordenad, sumamos o restamos columna o fila.
            
            match orientacion:
                case 'N':
                    fila2 = fila -1 
                    fila3 = fila -2            
                case 'S':
                   fila2 = fila + 1
                   fila3 = fila + 2
                case 'O':
                   columna2 = columna - 1
                   columna3 = columna - 2
                case 'E':
                    columna2 = columna + 1
                    columna3 = columna + 2


            if not (0 <= fila2 < filas_totales and 0 <= columna2 < columnas_totales):
                continue # Si la fila y columna no es mayor que las totlas y no son menores que 0, agregala a la variable segunda_pieza

            if not (0 <= fila3 < filas_totales and 0 <= columna3 < columnas_totales):
                continue # Si la fila y columna no es mayor que las totaless y no son menores que 0, agregala a la variable tercera_pieza
            segunda_pieza = (fila2,columna2)
            tercera_pieza = (fila3,columna3)

            if segunda_pieza in ocupadas:
                continue
            
            if tercera_pieza in ocupadas: # Compruebo que la tercera pieza no este ocupada antes.
                continue

            flota_grand.append([primera_pieza,segunda_pieza,tercera_pieza])
            ocupadas.add(primera_pieza)
            ocupadas.add(segunda_pieza)
            ocupadas.add(tercera_pieza)

            tablero[primera_pieza] = BARCO
            tablero[segunda_pieza] = BARCO
            tablero[tercera_pieza] = BARCO


            break
    return flota_grand,tablero





def flota_enorme_aleatoria(tablero, flota_peq,flota_med,flota_grand):
    filas_totales = tablero.shape[0]
    columnas_totales = tablero.shape[1] 

    ocupadas = set(flota_peq) 
    for barco in flota_med:
        for pieza in barco:
            ocupadas.add(pieza)
    for barco in flota_grand:
        for pieza in barco:
            ocupadas.add(pieza)
    flota_enorme = []
    random.seed(42)
    for barco in range(1):
        while True:
        # Primera pieza:
            fila = random.randint(0,filas_totales-1)
            columna = random.randint(0,columnas_totales-1)
            primera_pieza = (fila,columna)
        # Si la primera pieza es una de las ocupadas, vuelve a sacar una primera pieza
            if primera_pieza in ocupadas:
                continue
        # Elegimos orientacion, para asegurarnos que esten juntas las coordenadas de los barcos medianos.            
            orientacion = random.choice(['N','S','O','E'])

            fila2,columna2 = fila,columna # Muy importante ya que le decimos que la fila y columna de la segunda pieza en principio sea igual que la primera pieza y en funcion de la coordenad, sumamos o restamos columna o fila.
            fila3,columna3 = fila,columna
            fila4,columna4 = fila,columna
            match orientacion:
                case 'N':
                    fila2 = fila - 1
                    fila3 = fila - 2 
                    fila4 = fila - 3           
                case 'S':
                   fila2 = fila + 1
                   fila3 = fila + 2
                   fila4 = fila + 3
                case 'O':
                   columna2 = columna - 1
                   columna3 = columna - 2
                   columna4 = columna - 3
                case 'E':
                    columna2 = columna + 1
                    columna3 = columna + 2
                    columna4 = columna + 3

            if not (0 <= fila2 < filas_totales and 0 <= columna2 < columnas_totales):
                continue # Si la fila y columna no es mayor que las totaless y no son menores que 0, agregala a la variable segunda_pieza
            if not (0 <= fila3 < filas_totales and 0 <= columna3 < columnas_totales):
                continue # Si la fila y columna no es mayor que las totaless y no son menores que 0, agregala a la variable tercera_pieza
            if not (0 <= fila4 < filas_totales and 0 <= columna4 < columnas_totales):
                continue # Si la fila y columna no es mayor que las totaless y no son menores que 0, agregala a la variable tercera_pieza


            segunda_pieza = (fila2,columna2)
            tercera_pieza = (fila3,columna3)
            cuarta_pieza = (fila4,columna4)


            if segunda_pieza in ocupadas:
                continue
            if tercera_pieza in ocupadas: # Compruebo que la tercera pieza no este ocupada antes.
                continue
            if cuarta_pieza in ocupadas: # Compruebo que la tercera pieza no este ocupada antes.
                continue

            

            flota_enorme.append([primera_pieza,segunda_pieza,tercera_pieza,cuarta_pieza])
            ocupadas.add(primera_pieza)
            ocupadas.add(segunda_pieza)
            ocupadas.add(tercera_pieza)
            ocupadas.add(cuarta_pieza)

            tablero[primera_pieza] = BARCO
            tablero[segunda_pieza] = BARCO
            tablero[tercera_pieza] = BARCO
            tablero[cuarta_pieza] = BARCO


            break
    return flota_med,tablero



import pandas as pd
import numpy as np
import random 
def crear_tablero(filas=10, columnas=10):
  
    # Crea un tablero vacío usando NumPy.
    # Cada casilla empieza como AGUA (0).
 
    return np.full((filas, columnas), '-')


In [6]:
tablero_rival = crear_tablero(10, 10)


flota_peq, tablero_rival = flota_peq_aleatorio(tablero_rival)
flota_med, tablero_rival= flota_med_aleatoria(tablero_rival, flota_peq)
flota_grand, tablero_rival = flota_grand_aleatoria(tablero_rival, flota_peq, flota_med)
flota_gigante, tablero_rival = flota_enorme_aleatoria(tablero_rival, flota_peq, flota_med, flota_grand)

print(flota_peq)
print(flota_med)
print(flota_grand)
print(flota_gigante)


[(1, 0), (4, 3), (3, 2), (1, 8)]
[[(3, 3), (2, 3)], [(8, 3), (8, 4)], [(3, 7), (3, 6)]]
[[(8, 9), (7, 9), (6, 9)], [(4, 0), (5, 0), (6, 0)]]
[[(3, 3), (2, 3)], [(8, 3), (8, 4)], [(3, 7), (3, 6)]]
